# RAG Agent Creation Tutorial

## Overview
This notebook demonstrates how to create a **Retrieval-Augmented Generation (RAG) agent** using the Hyland Agent Platform API. You'll learn to:

1. **Authenticate** with the Hyland API using OAuth 2.0 client credentials
2. **Configure** a RAG agent with specific parameters
3. **Create** the agent via REST API calls
4. **Understand** the response and next steps

## Prerequisites
- Valid Hyland client credentials (`CLIENT_ID` and `CLIENT_SECRET`)
- Basic understanding of REST APIs and JSON
- Familiarity with Python (helpful but not required)

## Instructions
1. **Run cells sequentially** from top to bottom
2. **Read the explanations** before each code section
3. **Enter your credentials** when prompted (they will be hidden)
4. **Check the output** after each cell execution

---

## Step 1: Import Libraries and Define Authentication Function

The cell below imports the necessary Python libraries and defines a helper function for OAuth authentication.

**What this does:**
- **`json`**: For parsing API responses
- **`os`**: For accessing environment variables
- **`urllib`**: For making HTTP requests to the authentication endpoint
- **`requests`**: For making the main API calls
- **`getpass`**: For securely entering passwords (hides input)

**The `get_token()` function:**
- Takes client credentials as input
- Makes a POST request to Hyland's OAuth token endpoint
- Returns an access token for API authorization
- Handles errors gracefully

In [ ]:
import json
import urllib
import urllib.parse
import urllib.request
import requests
from getpass import getpass

def get_token(
    data: dict[str, str],
) -> str | None:
    encoded_data = urllib.parse.urlencode(data).encode("utf-8")

    headers_for_auth_request = {"Content-Type": "application/x-www-form-urlencoded"}

    auth_url = "https://auth.iam.dev.experience.hyland.com/idp/connect/token"
    print(f"Getting token from {auth_url}")
    try:
        req = urllib.request.Request(auth_url, data=encoded_data, headers=headers_for_auth_request, method="POST")

        with urllib.request.urlopen(req) as response_from_auth:            
            response_data = json.loads(response_from_auth.read().decode("utf-8"))
            access_token = response_data.get("access_token")
            print(f"access_token: {access_token}")
            return access_token

    except Exception as e:
        print(f"Failed to get token: {str(e)}")

    return None

## Step 2: Configure Your RAG Agent

The next cell defines the configuration for your RAG agent. This JSON payload specifies how your agent will behave.

**Key Configuration Options:**
- **`name`**: A friendly name for your agent ("DocumentHelper")
- **`description`**: What your agent does ("Document assistant")
- **`agentType`**: Set to "rag" for Retrieval-Augmented Generation
- **`limit`**: Maximum number of documents to retrieve (25)
- **`llm_model_id`**: The AI model to use (Amazon Nova Micro)
- **`system_prompt`**: Instructions that guide how the AI responds
- **`temperature`**: Controls creativity (0.7 = balanced)
- **`max_tokens`**: Maximum response length (4000 tokens)

**💡 Tip:** You can modify these values to customize your agent's behavior before running the cell.

In [ ]:
payload = {
    "name": "DocumentHelper",
    "description": "Document assistant",
    "agentType": "rag",
    "notes": "Initial version - shows all configuration options",
    "config": {
        "filter_value": {},
        "limit": 25,
        "llm_model_id": "amazon.nova-micro-v1:0",
        "system_prompt": "Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer:",
        "inference_config": {
            "temperature": 0.7,
            "max_tokens": 4000
        }
    }
}

## Step 3: Authenticate with Your Credentials

Now you'll enter your Hyland API credentials to get authorized access.

**What happens when you run this cell:**
1. **Environment Check**: First checks if credentials are available as environment variables
2. **Secure Input**: If not found, prompts you to enter them manually (input will be hidden)
3. **Validation**: Shows whether credentials were found or entered

**🔐 Security Note:** Your credentials are never displayed or stored in the notebook output.

**📝 If prompted, enter:**
- Your **Client ID** (looks like a UUID)
- Your **Client Secret** (a longer string)

In [ ]:
# Authentication - Enter your Hyland API credentials
print("Please enter your Hyland API credentials")
print("Your input will be hidden for security")

client_id = getpass("Enter your CLIENT_ID: ")
client_secret = getpass("Enter your CLIENT_SECRET: ")

if client_id and client_secret:
    print("Credentials entered successfully!")
else:
    print("Missing credentials - please run this cell again")

## Step 4: Get Access Token

This cell exchanges your credentials for an access token using OAuth 2.0 client credentials flow.

**What this does:**
- Calls the `get_token()` function with your credentials
- Uses the "client_credentials" grant type for server-to-server authentication
- Requests access to "hxp environment_authorization" scope
- Returns a temporary access token (typically valid for 1 hour)

**Expected output:** You should see the token endpoint URL and a success message with your access token.

In [ ]:
access_token = get_token({
        "client_id": client_id,
        "grant_type": "client_credentials",
        "client_secret": client_secret,
        "scope": "hxp environment_authorization"
    })

## Step 5: Prepare HTTP Headers

This cell sets up the HTTP headers needed for the API request to create your RAG agent.

**Headers explained:**
- **`Content-Type`**: Tells the server we're sending JSON data
- **`Accept`**: Tells the server we expect JSON response
- **`Authorization`**: Includes your Bearer token for authentication

**Note:** The `f'Bearer {access_token}'` syntax inserts your token from the previous step.

In [ ]:
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
  'Authorization': f'Bearer {access_token}'
}

## Step 6: Create Your RAG Agent

This is the main API call that creates your RAG agent on the Hyland platform.

**What this does:**
- Makes a POST request to the agents endpoint
- Sends your agent configuration (from Step 2) as JSON
- Includes authentication headers (from Step 5)
- Returns the creation result

**Expected responses:**
- **Status 201**: ✅ Success - Agent created successfully
- **Status 400**: ❌ Bad Request - Check your configuration
- **Status 401**: ❌ Unauthorized - Check your credentials
- **Status 403**: ❌ Forbidden - Check your permissions

**The response will include:**
- Your new agent's unique ID
- Confirmation of the configuration
- Any validation messages

In [ ]:
response = requests.post("https://api.agents.ai.dev.experience.hyland.com/agent-platform/v1/agents", json=payload, headers=headers)
print(response.status_code)
print(response.text)

## Next Steps & Troubleshooting

### ✅ If Successful (Status 201)
Your RAG agent has been created! You can now:
1. **Note your agent ID** from the response
2. **Use the agent** in your applications
3. **Test queries** against your document collection
4. **Monitor performance** through the Hyland dashboard

### ❌ Common Issues

**Status 400 (Bad Request)**
- Check your payload configuration
- Verify all required fields are present
- Ensure model ID is valid

**Status 401 (Unauthorized)**
- Verify your CLIENT_ID and CLIENT_SECRET
- Check if credentials have expired
- Ensure you're using the correct environment

**Status 403 (Forbidden)**
- Confirm your account has agent creation permissions
- Check if you're within usage limits

### 🔧 Modifying Your Agent
To create agents with different configurations:
1. Go back to **Step 2**
2. Modify the `payload` dictionary
3. Re-run from **Step 6** onwards

### 📚 Additional Resources
- [Hyland Agent Platform Documentation](https://docs.hyland.com)
- [RAG Agent Configuration Guide](https://docs.hyland.com/agents/rag)
- [API Reference](https://api.agents.ai.dev.experience.hyland.com/docs)